In [ ]:
import selenium
from selenium import webdriver  
import time  
import urllib  
from slugify import slugify
import requests
import random
import os

In [ ]:
"""
some variables needed to be set up by users
"""
conference = 'hri'
conference_url = "https://dl.acm.org/doi/proceedings/10.1145/1121241" # the conference url to download papers from
#see all available here: https://dl.acm.org/conference/hri/proceedings
chromedriver_path = 'chromedriver' # the chromedriver.exe path
year = '06'
root = './'+year+'/' #file path to save the downloaded papers

os.makedirs(root, exist_ok=True)
print(root)

In [ ]:
#setting up some driver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(executable_path='./chromedriver',chrome_options=chrome_options)

In [ ]:
wd.get(conference_url)

from retrieve_titles_urls_from_websites import *
retrieve = globals()['retrieve_from_'+conference]
print('Retrieving pdf urls. This could take some time...')
pdfurllist, pdfnamelist, df = retrieve(wd)


conf_name = wd.find_elements_by_class_name("left-bordered-title")[0].get_attribute('textContent')
#print(conf_name)
df['Conference'] = [conf_name]*df.shape[0]
df['Folder'] = [year]*df.shape[0]


In [ ]:
# download the papers one by one. The files are named after the titles (guaranteed to be valid file name after processed by slugify.)
print('Start downloading')
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:57.0) Gecko/20100101 Firefox/57.0'}
for i, url in enumerate(pdfurllist):
    if url != None :      
        pdfname = slugify(pdfnamelist[i])
        if os.path.isfile(root+'/'+pdfname+".pdf"):
            print('existed', i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
        else:
            print(i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
            data = requests.get(pdfurllist[i], timeout=80, headers=headers).content
            
            with open(root+'/'+pdfname+".pdf", 'wb')  as f:
                f.write(data)  
            _ = time.sleep(random.uniform(4,5)) # for anti-anti-crawler purpose

In [ ]:
df.to_csv(root+'/dataframe.csv', index = False) #creates dataframe